<a href="https://colab.research.google.com/github/woters/ds101/blob/master/llama_index_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install TruLens and Llama-Index

In [1]:
#!pip install trulens_eval llama_index llama-index-tools-yelp==0.1.2 trulens-providers-openai>=1.0.0 #langchain
#!pip install langchain langchain-core llama-index-core llama-index-tools-yelp #trulens-apps-llamaindex>=1.0.0

!pip install trulens trulens-apps-llamaindex trulens-providers-openai llama_index openai llama-index-tools-yelp

INFO: pip is looking at multiple versions of trulens-apps-langchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.3/600.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/13

In [2]:
#!pip install openai

In [3]:
import llama_index
from llama_index.agent.openai import OpenAIAgent
import openai

import os

In [4]:
# Set your API keys. If you already have them in your var env., you can skip these steps.

os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.environ["OPENAI_API_KEY"]

os.environ["YELP_API_KEY"] = "6RedxTkcNZswb6RaKC0pSWhyJtn_8ywEupkQu3cFl3bFEgTJO9iLZ28qR6Oo69HhcO8yHFRjb7M-I7w5aUJpmINK2Zl4wGm2QKlZViST0qfxhZPZjszUXMe7rzXbZXYx"
os.environ["YELP_CLIENT_ID"] = "ZzMGXB8TbxtRTleH_aIFZg"

# If you already have keys in var env., use these to check instead:
# from trulens_eval.keys import check_keys
# check_keys("OPENAI_API_KEY", "YELP_API_KEY", "YELP_CLIENT_ID")

### Set up our Llama-Index App

For this app, we will use a tool from Llama-Index to connect to Yelp and allow the Agent to search for business and fetch reviews.

In [5]:
# Import and initialize our tool spec
from llama_index.tools.yelp.base import YelpToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

# Add Yelp API key and client ID
tool_spec = YelpToolSpec(
    api_key=os.environ.get("YELP_API_KEY"),
    client_id=os.environ.get("YELP_CLIENT_ID")
)

In [6]:
gordon_ramsay_prompt = "You answer questions about restaurants in the style of Gordon Ramsay, often insulting the asker."

In [7]:
# Create the Agent with our tools
tools = tool_spec.to_tool_list()
agent = OpenAIAgent.from_tools([
        *LoadAndSearchToolSpec.from_defaults(tools[0]).to_tool_list(),
        *LoadAndSearchToolSpec.from_defaults(tools[1]).to_tool_list()
    ],
    verbose=True,
    system_prompt=gordon_ramsay_prompt
)

### Create a standalone GPT3.5 for comparison

In [8]:
client = openai.OpenAI()

chat_completion = client.chat.completions.create

In [9]:
from trulens_eval.tru_custom_app import TruCustomApp, instrument
from trulens_eval import TruCustomApp
from trulens_eval.tru_custom_app import instrument

class LLMStandaloneApp():
    @instrument
    def __call__(self, prompt):
        return chat_completion(
            model="gpt-3.5-turbo",
            messages=[
                    {"role": "system", "content": gordon_ramsay_prompt},
                    {"role": "user", "content": prompt}
                ]
        ).choices[0].message.content

llm_standalone = LLMStandaloneApp()

<ipython-input-9-b082f27626b3>:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.tru_custom_app import TruCustomApp, instrument
<ipython-input-9-b082f27626b3>:1: DeprecationWarning: The `trulens_eval.tru_custom_app` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.tru_custom_app import TruCustomApp, instrument


## Evaluation and Tracking with TruLens

In [10]:
# imports required for tracking and evaluation
from trulens_eval import Feedback, OpenAI, Tru, TruLlama, Select, OpenAI as fOpenAI
from trulens_eval.feedback import GroundTruthAgreement#, GroundeGroundTruthAgreementdness

tru = Tru()
# tru.reset_database() # if needed

<ipython-input-10-d55901198669>:3: DeprecationWarning: The `trulens_eval.feedback` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.feedback import GroundTruthAgreement#, GroundeGroundTruthAgreementdness


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


## Evaluation setup

To set up our evaluation, we'll first create two new custom feedback functions: query_translation_score and ratings_usage. These are straight-forward prompts of the OpenAI API.

In [11]:
class Custom_OpenAI(OpenAI):
    def query_translation_score(self, question1: str, question2: str) -> float:
        prompt = f"Your job is to rate how similar two quesitons are on a scale of 1 to 10. Respond with the number only. QUESTION 1: {question1}; QUESTION 2: {question2}"
        return self.generate_score_and_reason(system_prompt = prompt)

    def ratings_usage(self, last_context: str) -> float:
        prompt = f"Your job is to respond with a '1' if the following statement mentions ratings or reviews, and a '0' if not. STATEMENT: {last_context}"
        return self.generate_score_and_reason(system_prompt = prompt)

In [12]:
custom = Custom_OpenAI()

Now that we have all of our feedback functions available, we can instantiate them. For many of our evals, we want to check on intermediate parts of our app such as the query passed to the yelp app, or the summarization of the Yelp content. We'll do so here using Select.

In [13]:
# unstable: perhaps reduce temperature?

custom = Custom_OpenAI()
# Input to tool based on trimmed user input.
f_query_translation = Feedback(
    custom.query_translation_score,
    name="Query Translation") \
.on_input() \
.on(Select.Record.app.query[0].args.str_or_query_bundle)

f_ratings_usage = Feedback(
    custom.ratings_usage,
    name="Ratings Usage") \
.on(Select.Record.app.query[0].rets.response)

# Result of this prompt: Given the context information and not prior knowledge, answer the query.
# Query: address of Gumbo Social
# Answer: "
fopenai = fOpenAI()
# Question/statement (context) relevance between question and last context chunk (i.e. summary)
f_context_relevance = Feedback(
    fopenai.qs_relevance,
    name="Context Relevance") \
.on_input() \
.on(Select.Record.app.query[0].rets.response)

# Groundedness
#grounded = Groundedness(groundedness_provider=fopenai)


f_groundedness = Feedback(
    fopenai.groundedness_measure_with_cot_reasons, name = "Groundedness") \
.on(Select.Record.app.query[0].rets.response) \
.on_output()#.aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    fopenai.relevance,
    name="Answer Relevance"
).on_input_output()

✅ In Query Translation, input question1 will be set to __record__.main_input or `Select.RecordInput` .
✅ In Query Translation, input question2 will be set to __record__.app.query[0].args.str_or_query_bundle .
✅ In Ratings Usage, input last_context will be set to __record__.app.query[0].rets.response .
✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.app.query[0].rets.response .
✅ In Groundedness, input source will be set to __record__.app.query[0].rets.response .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Ground Truth Eval

It's also useful in many cases to do ground truth eval with small golden sets. We'll do so here.

In [14]:
golden_set = [
    {"query": "Hello there mister AI. What's the vibe like at oprhan andy's in SF?", "response": "welcoming and friendly"},
    {"query": "Is park tavern in San Fran open yet?", "response": "Yes"},
    {"query": "I'm in san francisco for the morning, does Juniper serve pastries?", "response": "Yes"},
    {"query": "What's the address of Gumbo Social in San Francisco?", "response": "5176 3rd St, San Francisco, CA 94124"},
    {"query": "What are the reviews like of Gola in SF?", "response": "Excellent, 4.6/5"},
    {"query": "Where's the best pizza in New York City", "response": "Joe's Pizza"},
    {"query": "What's the best diner in Toronto?", "response": "The George Street Diner"}
]

f_groundtruth = Feedback(
    GroundTruthAgreement(golden_set).agreement_measure,
    name="Ground Truth Eval") \
.on_input_output()

✅ In Ground Truth Eval, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth Eval, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Run the dashboard

By running the dashboard before we start to make app calls, we can see them come in 1 by 1.

In [15]:
tru.run_dashboard(
#     _dev=trulens_path, force=True  # if running from github
)

Starting dashboard ...


<ipython-input-15-32f881b2968e>:1: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard(


npm WARN exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://hot-cloths-know.loca.lt

  Submit this IP Address: 35.221.210.67



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [16]:
from trulens_eval import TruLlama

In [17]:
#!pip install trulens-apps-llamaindex>=1.0.0

### Instrument Yelp App

We can instrument our yelp app with TruLlama and utilize the full suite of evals we set up.

In [18]:
tru_agent = TruLlama(agent,
    app_id='YelpAgent',
    tags = "agent prototype",
    feedbacks = [
        f_qa_relevance,
        f_groundtruth,
        f_context_relevance,
        f_groundedness,
        f_query_translation,
        f_ratings_usage
    ]
)

In [19]:
tru_agent.print_instrumented()

Components:
	TruLlama (Custom) at 0x7d510a738950 with path __app__
	OpenAIAgent (Custom) at 0x7d510f35a410 with path __app__.app
	ChatMemoryBuffer (Custom) at 0x7d510e11d4e0 with path __app__.app.memory
	SimpleChatStore (Custom) at 0x7d510e976fc0 with path __app__.app.memory.chat_store

Methods:
Object at 0x7d510e11d4e0:
	<function BaseChatStoreMemory.put at 0x7d51102d27a0> with path __app__.app.memory
	<function BaseMemory.put at 0x7d51102d1ea0> with path __app__.app.memory
Object at 0x7d510f35a410:
	<function AgentRunner.chat at 0x7d510eaee7a0> with path __app__.app
	<function AgentRunner.stream_chat at 0x7d510eaee440> with path __app__.app
	<function AgentRunner.achat at 0x7d510eaeed40> with path __app__.app
	<function AgentRunner.astream_chat at 0x7d510eaef0a0> with path __app__.app
	<function BaseQueryEngine.query at 0x7d51108a23b0> with path __app__.app
	<function BaseQueryEngine.aquery at 0x7d51108a32e0> with path __app__.app
	<function BaseQueryEngine.synthesize at 0x7d51108a33

### Instrument Standalone LLM app.

Since we don't have insight into the OpenAI innerworkings, we cannot run many of the evals on intermediate steps.

We can still do QA relevance on input and output, and check for similarity of the answers compared to the ground truth.

In [20]:
tru_llm_standalone = TruCustomApp(
    llm_standalone,
    app_id="OpenAIChatCompletion",
    tags = "comparison",
    feedbacks=[
        f_qa_relevance,
        f_groundtruth
    ]
)

In [21]:
tru_llm_standalone.print_instrumented()

Components:
	TruCustomApp (Custom) at 0x7d51083b16c0 with path __app__
	LLMStandaloneApp (Custom) at 0x7d510e336680 with path __app__.app

Methods:
Object at 0x7d510e336680:
	<function LLMStandaloneApp.__call__ at 0x7d510b2b1cf0> with path __app__.app


### Start using our apps!

In [22]:
prompt_set = [
    "What's the vibe like at oprhan andy's in SF?",
    "What are the reviews like of Gola in SF?",
    "Where's the best pizza in New York City",
    "What's the address of Gumbo Social in San Francisco?",
    "I'm in san francisco for the morning, does Juniper serve pastries?",
    "What's the best diner in Toronto?"
]

In [23]:
for prompt in prompt_set:
    print(prompt)

    with tru_llm_standalone as recording:
        llm_standalone(prompt)
    record_standalone = recording.get()

    with tru_agent as recording:
         agent.query(prompt)
    record_agent = recording.get()

What's the vibe like at oprhan andy's in SF?
Added user message to memory: What's the vibe like at oprhan andy's in SF?
=== Calling Function ===
Calling function: business_search with args: {"location":"San Francisco","term":"Orphan Andy's"}
Got output: Content loaded! You can now search the information using read_business_search

=== Calling Function ===
Calling function: read_business_search with args: {"query":"What is the vibe like at Orphan Andy's in San Francisco?"}
Got output: The vibe at Orphan Andy's in San Francisco can be described as a classic American diner experience with a focus on breakfast and brunch offerings.

What are the reviews like of Gola in SF?
Added user message to memory: What are the reviews like of Gola in SF?
=== Calling Function ===
Calling function: business_search with args: {"location":"San Francisco","term":"Gola"}
Got output: Content loaded! You can now search the information using read_business_search

=== Calling Function ===
Calling function: read